In [9]:
import gym
import numpy as np 
import matplotlib.pyplot as plt
import copy

NUM_EP=20000
LR=0.000025
GAMMA=0.99

env=gym.make('CartPole-v0')
actionspace=env.action_space.n
np.random.seed(1)

w=np.random.rand(4,2)

def policy(state,w):
    z=state.dot(w)
    exp=np.exp(z)
    return exp/np.sum(exp)

def grad_soft(softmax):
    s=softmax.reshape(-1,1)
    return np.diagflat(s)-np.dot(s,s.T)

def select_action(actionspace,probs):
    return np.random.choice(actionspace,p=probs[0])

def update(w,grads,rewards,LR,GAMMA):
    for e in range(len(grads)):
        w+=LR*grads[e]*sum([r*(GAMMA**r)for t,r in enumerate(rewards[e:])])
    return w 


def next_episde(w,state,actionspace):
    probs=policy(state,w)
    action=select_action(actionspace,probs)
    statee,reward, done,_ = env.step(action)
    statee=statee[None,:]
            
            
    dsoft=grad_soft(probs)[action,:]
    log=dsoft/ probs[0,action]
    delo=state.T.dot(log[None,:])
    return delo,reward,statee,done

def train(w,actionspace,NUM_EP,GAMMA,LR):
   
    for i in range(NUM_EP):
        grads=[]
        rewards=[]
        ep_rewards=[]
        score=0
        state=env.reset()[None,:]
        
        
        while True:
            delo, reward,statee, done=next_episde(w,state,actionspace)
            grads.append(delo)
            rewards.append(reward)
        
            score+=reward
            state=statee
            if done:
                break
                #env.reset()
        w=update(w,grads,rewards,LR,GAMMA)
        ep_rewards.append(score)
        print("EP: " + str(i)+ " Score: " +"Weight:" +str(w)+str(score) + "         ",end="\r", flush=False) 
        #rint(w)
        env.close()
    
state = env.reset()[None,:]
train(w,actionspace,NUM_EP,GAMMA,LR)

## Reinforcement learning introduction: Vanilla Policy Gradient 
![alt text](reinfo.jpg)
The basic idea behind deep reinforcement learning is that an agent in an envionment with a given state has a policy, which is typically a convolutional neural net that predicts the probability of the next action, then the as agent observes the next state the parameters of the policy can be slowly learned. The agent learns which actions are "good"  by the rewards recieved by taking a particular action in a certain state.  Mathematically the trajectory of the agent is given by the sequence of the state and action over an "episode". The goal of policy gradient is to train our policy parameters by running Monte Carlo simulations over many episodes in order to maximize to discounted expected reward. The vanilla policy gradient sets the stage for classes of more advanced RL algorthims such as actor critic and Q-learning methods. 
\begin{array}{rcl}
Max_\theta E_\pi R(T)
\end{array}
The parameters of the policy are learned via gradient descent 
\begin{array}{rcl}
\theta_{t+1} &=& \theta_t +\alpha \nabla_\theta E_\pi R(T)
\end{array}
by exploiting properties of logarithm and simple algebraic manipulation the gradient of the expected reward is given by:
\begin{array}{rcl}    
\nabla_\theta E_\pi R(T) &=& E_\pi \nabla_\theta log (P(t | \theta) R(T))
\end{array}
in a Makrov decision process the conditional probability over a trajectory is given by 
\begin{array}
(P(t | \theta)=p(s_0) \prod_0^{T-1} p(s_{t+1}|s_t,a_t) \pi_{\theta}(a_t,s_t)
\end{array}
This yields the equation for the gradient of the expected reward for Monte Carlo sampling:
\begin{array}
\\ \nabla_\theta E_\pi R(T) &=& \frac{1}{M} \Sigma_T \Sigma_{t=0}^{T-1} log  \pi_\theta R(T)
\end{array}
This was not a formal derivation, all the details of the derivation can be found in the openAI docs. Instead of using a convolutional neural net we will keep it simple and use a simple logistic regression model as our Pi_theta.

Tutorial: https://www.janisklaise.com/post/rl-policy-gradients/

Another great tutorial: http://quant.am/cs/2017/08/07/policy-gradients/

ANother tutorial: https://medium.com/samkirkiles/reinforce-policy-gradients-from-scratch-in-numpy-6a09ae0dfe12

derivation: https://spinningup.openai.com/en/latest/spinningup/rl_intro3.html#deriving-the-simplest-policy-gradient

Likely the best notes on the subject matter, CS 285 from Berkley( would recommend the whole course); there are notes from one of researchers who discovered the PPO algorthim in 2016 Sergey Levine: https://rail.eecs.berkeley.edu/deeprlcourse/static/slides/lec-5.pdf

The main issue with the policy gradient is that it has a rather high varinace and hence has a slow convergence rate taking often many iterations. It is also difficult to select hyperparameters in the optimization process such as the learning rate, for particular optimization algorthims it is certainly easier, for example in the Berkley notes Levin mentions that Adam's step size is not bad.

In [10]:
state=env.reset()[None,:]

In [ ]:
# note this is the W i obtained from only 5000 iterations it requires 20,000 to fully learn
while True:
    w=[[ 1.06721556,0.07013094],    
 [-0.71238789,1.01483483],
 [-1.22409825,1.46319274],
 [-2.51589573,3.04771667]]
    #env.render()
    probs=policy(state,w)
    action=select_action(actionspace,probs)
    statee,reward, done,_ = env.step(action)
    statee=statee[None,:]
    env.render()
    env.close

/opt/anaconda3/lib/python3.7/site-packages/gym/envs/classic_control/cartpole.py:151: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  "You are calling 'step()' even though this "
